# IMPORT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
import pymysql
import paramiko
from sshtunnel import SSHTunnelForwarder

#from datetime import datetime, date, timedelta

pkeyfilepath = '/Users/nagyeonghun/Desktop/ssh_key.pem' ######################### ssh_key.pem 파일 경로 넣으시면 됩니다.
password = 'gitple5293'
mypkey = paramiko.RSAKey.from_private_key_file(pkeyfilepath, password=password)
sql_hostname = 'db-8j8d2-fkr.cdb.fin-ntruss.com'
sql_username = 'rudgnsdl06'            ######################### sql_username 입력하시면 됩니다.
sql_password = 'Z2st8puvos&39ipaSibu'            ######################### sql_password 입력하시면 됩니다.
sql_main_database = 'betterday_db'
sql_port = 3306
ssh_host = '10.0.1.14'
ssh_user = 'rudgnsdl06'                ######################### ssh_user 입력하시면 됩니다.
ssh_port = 22

# soslSuregi97o#rop0n0

# 적요 기반 유저 소득 추정
- pani에서 작업한 BANK_42 ~ BANK_47(요구불계좌 총 급여 입금금액 M0 ~ M5) 사용
- TRANS_MEMO에 '급여'라고 적힌 내역을 확인하는 방식

In [3]:
def find_user_income(user_id=268):
    with SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_pkey=mypkey,
            remote_bind_address=(sql_hostname, sql_port)) as tunnel:
            con = pymysql.connect(host='127.0.0.1', user=sql_username,
                                  passwd=sql_password, db=sql_main_database,
                                  port=tunnel.local_bind_port)

            BCID = user_id
            sql = f"""
            SELECT
             CSS_BANK_42({BCID})
            ,CSS_BANK_43({BCID})
            ,CSS_BANK_44({BCID})
            ,CSS_BANK_45({BCID})
            ,CSS_BANK_46({BCID})
            ,CSS_BANK_47({BCID});
            """
            temp = pd.read_sql(sql, con)
            temp = temp.dropna(axis=1)

    return temp

In [4]:
find_user_income(268)

,CSS_BANK_42(268),CSS_BANK_43(268),CSS_BANK_44(268),CSS_BANK_45(268),CSS_BANK_46(268),CSS_BANK_47(268)
0,0,13331060,6344520,6365470,7685470,7025470


# 입금 내역 기반 유저 소득 추정

In [11]:
def find_user_transamt(user_id=268):
    with SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_pkey=mypkey,
            remote_bind_address=(sql_hostname, sql_port)) as tunnel:
            con = pymysql.connect(host='127.0.0.1', user=sql_username,
                                  passwd=sql_password, db=sql_main_database,
                                  port=tunnel.local_bind_port)

            BCID = user_id
            sql = f"""
                SELECT ID, BANK_ACCOUNTS_ID, TRANS_DTIME, DATE_FORMAT(TRANS_DTIME, "%Y-%m-%d") AS DATE, TRANS_TYPE, TRANS_AMT, BALANCE_AMT
                FROM MD_BANK_ACCOUNTS_DEPOSIT_TRANS_HIST
                WHERE BANK_ACCOUNTS_ID IN (SELECT ID FROM MD_BANK_ACCOUNTS WHERE TOKEN_ID IN (SELECT ID FROM MD_USER_TOKEN WHERE BDAY_CUST_ID = {BCID})
                        AND IS_CONSENT =1 AND ACCOUNT_TYPE = 1001 AND ACCOUNT_STATUS=1)
                AND DATE_FORMAT(TRANS_DTIME, "%Y-%m")
                    BETWEEN (SELECT DATE_FORMAT(DATE_SUB(CRT_DT, INTERVAL 6 MONTH), "%Y-%m") FROM BDAY_CUST_LOGIN_HIST bclh WHERE CUST_ID = {BCID} LIMIT 1) AND (SELECT DATE_FORMAT(DATE_SUB(CRT_DT, INTERVAL 1 MONTH), "%Y-%m") FROM BDAY_CUST_LOGIN_HIST bclh WHERE CUST_ID = 268 LIMIT 1)
                ORDER BY TRANS_DTIME;
            """
            temp = pd.read_sql(sql, con)
    if not temp.empty:        
        temp = temp.astype({'TRANS_AMT':'float', 'BALANCE_AMT':'float'})
        temp = temp.astype({'TRANS_AMT':'int', 'BALANCE_AMT':'int'})
        temp['DATE'] = pd.to_datetime(temp['DATE'])

        #temp에서 자기계좌송금 내역 제거
        temp = temp.drop(temp[(temp['TRANS_DTIME']-temp['TRANS_DTIME'].shift(1) < timedelta(seconds=60))&(temp['TRANS_AMT'].duplicated())].index).reset_index(drop=True)

        temp_03 = temp[(temp.TRANS_TYPE=='03')&(temp.TRANS_AMT>=1500000)].reset_index(drop=True)
        df_03 = pd.DataFrame(index=pd.date_range(start=temp['DATE'].apply(lambda x: x.strftime('%Y-%m')).min(), end=temp['DATE'].apply(lambda x: (x+relativedelta(months=1)).strftime('%Y-%m')).max(), inclusive='left'), columns=temp_03['BANK_ACCOUNTS_ID'].unique())
        temp_03_last = temp_03.groupby(['DATE', 'BANK_ACCOUNTS_ID'])['TRANS_AMT'].last() #이부분이 first, last, sum, mean,,,, 뭘로할지 참 애매

        #df_03 채우기
        for date in pd.date_range(start=temp_03['DATE'].min(), end=temp_03['DATE'].max()):
            for acc in temp_03['BANK_ACCOUNTS_ID'].unique():
                try:
                    df_03.loc[date, acc] = temp_03_last[date][acc]
                except:
                    continue

#         M = []
#         for i in df_03:
#             L = []
#             for month in range(start_month, end_month+1):
#                 monthly_arr = df_03.loc[df_03.index.month==month, i].values
#                 monthly_arr = np.pad(monthly_arr, (2, 2+31-monthly_arr.size), constant_values=np.nan) #5일씩 볼거라 패딩 2씩 더해줌
#                 L.append(monthly_arr)
#             acc_arr = np.stack(L)
#             M.append(acc_arr)
#         total_arr = np.stack(M)

    else:
        print('there is no result of query !')
        return None

    return df_03

In [12]:
find_user_transamt()

,1562,1565,1564,1567
2022-04-01,NaN,NaN,NaN,NaN
2022-04-02,NaN,NaN,NaN,NaN
2022-04-03,NaN,NaN,NaN,NaN
2022-04-04,NaN,NaN,NaN,NaN
2022-04-05,NaN,NaN,NaN,NaN
...,...,...,...,...
2022-09-26,NaN,NaN,NaN,NaN
2022-09-27,NaN,NaN,NaN,NaN
2022-09-28,NaN,NaN,NaN,NaN
2022-09-29,NaN,NaN,NaN,NaN


## median
total_arr
- shape : k(통장 개수) x 6(or 12) x 31

filter
- shape : 1 x 6(or 12) x 5(3 to 7)

In [17]:
def median_filter(df, filter_size=(4, 6, 5), stride=1):
    
    start_month = df.index.month.min()
    end_month = df.index.month.max()
        
    M = []
    for i in df:
        L = []
        for month in range(start_month, end_month+1):
            monthly_arr = df.loc[df.index.month==month, i].values
            monthly_arr = np.pad(monthly_arr, (2, 2+31-monthly_arr.size), constant_values=np.nan) #5일씩 볼거라 패딩 2씩 더해줌
            L.append(monthly_arr)
        acc_arr = np.stack(L)
        M.append(acc_arr)
    arr = np.stack(M)
    
    arr = arr.astype('float64')
    arr_shape = np.shape(arr)
    result_shape = tuple(np.int64((np.array(arr_shape)-np.array(filter_size))/stride+1))

    result_med = np.zeros(result_shape[-1])
    result_mod = np.zeros(result_shape[-1])
    result_near = np.zeros(result_shape[-1], dtype='int64')
    result_std = np.zeros(result_shape[-1])
    
    for d in range(0, result_shape[-1], stride):
        tmp = arr[:, :, d:d+filter_size[-1]]
        tmp = tmp.ravel() #isnotnull 개수로 값 몇개 잡힌지는 볼 수 있음, rounding
        med = np.nanmedian(tmp) # med값이랑 med값의 빈도든 뭐든 빈도를 봐야함
        mod = stats.mode(tmp, nan_policy='omit')[0]
        
        #tmp에서 med 근처에 있는 애들 개수 셀거야
        near = np.count_nonzero(~np.isnan(np.where((med*0.9<=tmp)&(tmp<med*1.1), tmp, np.nan)))
        
        #std 고려해서 결정할거야
        std = np.nanstd(tmp)
        
        #iqr 뺄거야
        
        #rounding(binning) 해서 개수 셀거야


        result_med[d] = med
        result_mod[d] = mod #mod는 1개여도 잡혀서 mod말고 개수를 파악해야될듯
        # 그리고 mod는 일치하는 값에 대해서만 최빈을 구함, 하지만 완전히 일치하지 않더라도 비슷한 값에 대해 개수를 파악할 필요있음
        result_near[d] = near
        result_std[d] = std
        
    result = result_med[np.argmax(result_near)]


    return result

In [18]:
median_filter(find_user_transamt())

6354995.0

# 소득 추정 최종 함수

In [23]:
def predict_income(BCID):
    
    df = find_user_income(BCID)
    
    monthly_incomes = [int(val) for val in df.T.values]
    
    
    arr = df.T.value_counts()
    mask1 = arr[arr>=2]
    
    # 1. 동일한 값이 있으면 그 값을 소득으로 (동일한 값의 개수 2개로 할지 3개로 할지 결정해야함)
    if not mask1.empty:
        # 1.1. 한 개의 값만 동일할 경우 그 값을 소득으로
        if len(mask1) == 1:
            return mask1.idxmax()[0]
        # 1.2. 여러 개의 값이 동일할 경우 그 중 최근 값을 소득으로
        else:
            min_month = 13
            for income_pred in mask1.index:
                for i, val in enumerate(monthly_incomes):
                    if val == income_pred[0]:
                        min_month = min(min_month, i)
            return monthly_incomes[min_month]
    # 2. 동일한 값이 없으면
    else:
        L = []
        # 6개 값을 서로 비교하여 대표값 설정
        for monthly_income in monthly_incomes:
            count = 0
            for comparison in monthly_incomes:
                if (monthly_income*0.95 <= comparison) and (comparison < monthly_income*1.05):
                    count += 1
            
            L.append(count)

        if L:
            # 비교했을 때 서로 비슷한 값이 하나라도 있다면
            if max(L) != 1:
                return monthly_incomes[L.index(max(L))]
            else:
                # 6개월 중 적요가 한 번만 잡히거나 잡힌 모든 값이 극도로 다른 경우
                return monthly_incomes.mean()
        
        else:
            # 적요로 잡히지 않는 경우
            df = find_user_transamt(BCID)
            pred = median_filter(df)
            return pred
            

In [24]:
predict_income(268)

6344520